## Trabajo Fin de Máster <br/> Diseño de una arquitectura multimodal para descripción textual de pares imagen-audio

## Script 1. Preprocesamiento de audio usando PyTorch

En este notebook, creamos una base de datos para un clasificador de audio, a partir de unos pocos audios originales. Para ello, usamos lo compartido en el artículo de [TowardsDataScience](https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5). Tomamos las clases que se definen en dicho artículo, y las enriquecemos con métodos propios, y otros extraídos de esta entrada en [medium](https://medium.com/@alibugra/audio-data-augmentation-f26d716eee66).


### Paso 1. Montamos el almacenamiento

Comenzamos dando permiso a Drive para acceder a mis archivos y nos situamos en la carpeta donde se halla la carpeta con notebooks y la carpeta con nuestras librerías para tratar los datos.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/TFM/Proyecto/Scripts

Mounted at /content/drive
/content/drive/MyDrive/TFM/Proyecto/Scripts


### Paso 2. Lectura de audios originales

Accedemos a la carpeta donde se encuentran los audios originales y generamos un dataframe con información básica sobre ellos.

In [ ]:
import os
import shutil
import pandas as pd

In [ ]:
from tfm_lib.audio_processing import AudioUtil, AudioAugmentation

def audio_files_df(database_path):

    participants = os.listdir(database_path)
    if '.DS_Store' in participants:
        participants.remove('.DS_Store')

    rel_paths = []
    sample_rates = []
    channels = []
    duration = []
    classID = []

    for person in participants:
        relative_path = f"{database_path}/{person}/audio.ogg"
        sig, sr = AudioUtil.open(relative_path)

        rel_paths.append(relative_path)
        sample_rates.append(sr)
        channels.append(sig.shape[0])
        duration.append(sig.shape[1]/sr)
        classID.append(person)

    return pd.DataFrame({
        "relative_path": rel_paths,
        "sample_rate": sample_rates,
        "channels": channels,
        "duration (s)": duration,
        "classID": classID
    })


df = audio_files_df('../Database_mini')
display(df)

,relative_path,sample_rate,channels,duration (s),classID
0,../Database_mini/Jose Alberto Azorin Puche/aud...,48000,1,22.893500,Jose Alberto Azorin Puche
1,../Database_mini/Noelia Sanchez Alonso/audio.ogg,48000,1,20.413500,Noelia Sanchez Alonso
2,../Database_mini/Maria Jose Morales Forte/audi...,48000,1,21.873500,Maria Jose Morales Forte
3,../Database_mini/Genesis Reyes Arteaga/audio.ogg,48000,1,24.677833,Genesis Reyes Arteaga
4,../Database_mini/Juan Cuesta Lopez/audio.ogg,48000,1,21.937833,Juan Cuesta Lopez
5,../Database_mini/Juanjo Bautista Ibanez/audio.ogg,48000,1,33.073500,Juanjo Bautista Ibanez


### Paso 3. Creación de la base de datos de audio

En este paso, leemos cada uno de los audios que conocimos anteriormente y aplicamos sobre ellos una serie de transformaciones para homogeneizarlos, y también para crear audios nuevos a partir de los originales.

Las transformaciones son las siguientes:

* **Limpieza.** Podamos el inicio y final del audio, puesto que la mayoría de participantes comienzan y acaban su grabación con unos instantes de silencio (no nos interesa que la red aprenda silencios que no formen parte del discurso leído).

* **Resampleado.** Nos aseguramos de que la frecuencia de muestreo de todos los audios originales sea la misma (modificándola en caso de que sea necesario).

* **Igualar número de canales.** Dado un número deseado de canales (1 ó 2), nos quedamos con el primer canal (para pasar de estéreo a mono) o duplicamos el canal existente (de mono a estéreo).

* **Fragmentación.** Cortamos el audio en una serie de trozos de una duración determinada. En este caso se multiplica el número de audios de 5 a 10 veces.

Una vez tenemos estos trozos de audios originales, generemos audios completamente nuevos con técnicas de Data Augmentation:

* **Inserción de ruido.** Distorsionamos de forma aleatoria la señal.

* **Tralación temporal.** Pegamos el inicio y final del audio y trasladamos el inicio a un instante distinto del original.

Por medio de estas trasnformaciones, multiplicamos por 5 el número de audios que habíamos obtenido en la fragmentación.

In [ ]:
def ExtendDatabase(db_path, new_db_path, sample_rate, channels, duration):

    # For each name in the list of participants, we apply the same transformations
    # and create a new folder in the new database with all the new audios
    participants = os.listdir(db_path)
    participants.remove(".DS_Store")

    classID = []
    rel_paths = []
    for person in participants:
        relative_path = f"{db_path}/{person}/audio.ogg"
        aud = AudioUtil.open(relative_path)

        # First, we clean and standarize the audios
        cln_aud = AudioUtil.prune_audio(aud)
        reaud = AudioUtil.resample(cln_aud, sample_rate)
        rch_aud = AudioUtil.rechannel(reaud, channels)

        # Now, we split the audio that results and save the results
        lst_aud_0 = AudioUtil.split_audio(rch_aud, duration=duration)
        lst_aud_1 = list(map(AudioAugmentation.add_noise, lst_aud_0))
        lst_aud_2 = list(map(AudioAugmentation.time_shift, lst_aud_0))
        lst_aud_3 = list(map(AudioAugmentation.add_noise, lst_aud_2))
        lst_aud_4 = list(map(AudioAugmentation.time_shift, lst_aud_0))
        lst_aud_5 = list(map(AudioAugmentation.add_noise, lst_aud_4))

        # Remove the new database folder in case it exists
        # Create the empty folder with the same name
        folder_path = f'{new_db_path}/audio/{person}'
        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path)

        for k in range(6):
            var_name = f"lst_aud_{k}"
            for i, audio in enumerate(vars()[var_name]):
                path = f'{folder_path}/audio{str(k).zfill(2)}{str(i).zfill(2)}.ogg'
                _ = AudioUtil.save(audio, path)
                classID.append(person)
                rel_paths.append(path)

    return pd.DataFrame({
        'audio_path': rel_paths,
        'classID': classID
        })

database_df = ExtendDatabase('../Database_mini', '../Final_Database_mini', 48000, 1, 4)
database_df.to_csv("../Final_Database_mini/audio/audioDB.csv", index=False)

In [ ]:
database_df = pd.read_csv("/content/drive/MyDrive/TFM/Proyecto/Final_Database_mini/audio/audioDB.csv")
database_df.head()

,audio_path,classID
0,../Final_Database_mini/audio/Jose Alberto Azor...,Jose Alberto Azorin Puche
1,../Final_Database_mini/audio/Jose Alberto Azor...,Jose Alberto Azorin Puche
2,../Final_Database_mini/audio/Jose Alberto Azor...,Jose Alberto Azorin Puche
3,../Final_Database_mini/audio/Jose Alberto Azor...,Jose Alberto Azorin Puche
4,../Final_Database_mini/audio/Jose Alberto Azor...,Jose Alberto Azorin Puche
